In [1]:
#This script reads a textfile created by Adrian's Scripts (Whitney and Bobby) containing NextBus data pulls on 
#vehicle locations and stop data

In [103]:
def importLocData(filename):
    #todo: turn this into a function that takes in a route# and date to generate textfile name

    import numpy as np
    import json
    import pprint

    strng = open(filename).read()

    array1 = strng.split('[')      #split file string into rows
    array1 = np.asarray(array1)    #turns rows into numpy array format
    array1 = array1[2:]            #removes leading brackets todo: do this in one line(embed in above line)


    #Create an array of columns from rows
    data_arr = []

    for datapt in array1:
        #split the datapts into a subarrays, strip out quotn marks and convert to correct format
        subarr = datapt.split(',')
        newarr=[0,0,0,0,0]
        if len(subarr) > 5:
            newarr[0] = float(subarr[0][1:-1])
            newarr[1] = float(subarr[1][2:-1])
            newarr[2] = float(subarr[2][2:-1])
            newarr[3] = float(subarr[3][2:-1])        
            newarr[4] = float(int((int(subarr[5][1:14])/1000)-int(subarr[4][2:-1])))
            data_arr.append(newarr)

    #sort array of processed data 
    ###NB. 'sorted' method only works on List objects
    from operator import itemgetter, attrgetter, methodcaller

    data_arr = sorted(data_arr, key=itemgetter(0,4))
    return data_arr

In [104]:
def routeOrientation(data_arr):
    #convert data_arr list object into numpy array
    import numpy as np
    nparr=np.asarray(data_arr)

    maxCoords = np.amax(nparr, axis=0)[1:3]
    minCoords = np.amin(nparr,axis=0)[1:3]

    #c is the overall max displacement, N-S, and E-W
    displacement = maxCoords-minCoords

    # ratio for orientation
    dRat = displacement[0]/displacement[1]

    #arbitrarily chose 2:3 ratio for my rectangle....hope this logic holds =D
    if dRat < 0.6667 :
        orientation = 'EW'
    elif dRat > 1.5 :
        orientation = 'NS'
    else :
        print('EW or NS?')
        #ask user for orientation
    return orientation

In [105]:
def importStopData(stopsFilename):
    #aka ...there's a script named "Bobby", read this data and turn it into something useable

    stopsarr = [[43.65381, -79.4635860],[43.65663, -79.34890],[43.65837, -79.33896], [43.66155, -79.32947]]
    return stopsarr

In [106]:
def stopList(stopsarr):
    #todo: convert list of floating point coords into a list of strings
    #just used for comparison in filtering data function, could probably be rewritten to compare floating point values
    #the code just looked cleaner this way =P
    stops = ["(43.65381, -79.4635860)","(43.65663, -79.34890)","(43.65381, -79.35860)", "(43.66155, -79.32947)","(43.641182, -79.434135)"]
    return stops

In [107]:
def extrapolateT(t1,t2, d1,d2,stop):
    delta_t = t2-t1
    delta_d = d2-d1
    delta_stop = stop-d1
    
    extrapT = (delta_stop/delta_d)*delta_t + t1
    
    return extrapT

In [108]:
## debugging a function:
## x = extrapolateT(1514924740.0, 1514924416.0, -79.445831, -79.485619, -79.463586)
## print(x)

In [109]:
def assignStop(datapt, orientation, stops_arr):
    import numpy as np    
    n = np.shape(stops_arr)[0]
    
    stop = 0
    if (orientation == 'EW') and (datapt[3] in range (90,270)):
        stop = n
    elif (orientation =='NS') and (datapt[3] < 180) :
        stop = n
    return stop    

In [153]:
def prepData(data_arr, stops_arr, stops):
    #Filtering data
    import math
    import numpy as np
    nparr=np.asarray(data_arr)

    data_filt =[]
    n = np.shape(nparr)[0]
    thr = 0.000001
    
    orientation = routeOrientation(data_arr)
    if orientation =='NS' :
        dir = 0
    elif orientation == 'EW' :
        dir = 1
    else :
        print('Orientation error')
        
    ct_loc = 1

    #remove datapoints from same location/veh
    for i in range (0, np.shape(nparr)[0]-1):
        #check if loc and veh are the same
        lat = nparr[i][1]
        long = nparr[i][2]
        newarr=[0,0,0,0,0,0]
        
        if (abs(nparr[i][0]-nparr[i+1][0])>=thr) :
            # ^^ if it's a 'new' veh
            # todo: write a function to find the nearest stop, once a new vehicle enters the network
            # ideally, it will either be the first or last stop, I'm not sure
            # another option would be to just assign the first/last stop based on the route direction and veh heading
            # lots of options! 
            stopNum = assignStop(nparr[i], orientation, stops_arr)
            newarr = np.append(nparr[i],stopNum)
            
        #filter to remove duplicate values (we only care about the veh arrival at a location)
        elif ((abs(lat-nparr[i+1][1])>=thr)or (abs(long-nparr[i+1][2])>=thr)):
            #i.e. if the lat, long, or veh number has changed: continue, else: ignore this point
            ###note for ^^: computers don't always store 'float' types accurately (e.g. 1.0 could really be 1.000004594852)
            ###:. when comparing them, use a threshold like abs(1.0 - 1.0) <= 0.0001

            #filter for datapoints where either the veh is at a stop or just passed one 
            coords = "("+str(lat)+", "+str(long)+")"
            if coords in stops :
                #if it falls at a bus stop -> add
                #todo: this part might be redundant, see how to eliminate this
                stopNum = stops.index(coords)
                newarr = np.append(nparr[i],stopNum)
                print("{} at the stop".format(i))              
            else :                
                #   else if it just passed a bus stop -> estimate time it passed the stop and add a line                
                for j in range(0,np.shape(stops_arr)[0]-1):
                    if ((stops_arr[j][dir]-nparr[i][dir+1])*(stops_arr[j][dir]-nparr[i+1][dir+1])) <0:
                    #i.e. if the stop is between datapts (aka, one greater than and one less than) continue, else: ignore
                    #todo: right now, it checks if a point is near any stop. Let's find a way to sort the stops
                        # and start by checking the stops by the datapoint[i]!
                        estT = extrapolateT(nparr[i][4], nparr[i+1][4], nparr[i][1+dir], nparr[i+1][1+dir], stops_arr[j][dir])
                        print('passed the stop{},{},,{}'.format(nparr[i][1+dir],nparr[i+1][1+dir],stops_arr[j][dir]))
                        newarr = [nparr[i][0],stops_arr[j][0], stops_arr[j][1],nparr[i][3],estT,j]
                        break
                #print("no stop, discard pt")
                ct_loc = ct_loc+1

            #newarr = np.append(nparr[i],stopNum)

        if newarr[0] > 1 :
            data_filt.append(newarr)            
        else:        
            ct_loc = ct_loc+1

    i = i + 1

    return data_filt

In [154]:
def main():
    #import data
    data_arr = importLocData('NextBusOutput-Route501-02Jan2018-TuePM.txt') #file output from Whitney
    stops_arr = importStopData('file output from Bobby')
    stops = stopList(stops_arr)
    
    #prepare data for processing
    orientation = routeOrientation(data_arr)
    #print(data_arr[3])
    #print(data_arr[4])
    clean_arr = prepData(data_arr, stops_arr, stops)
    import numpy as np
    
    print(np.shape(clean_arr))
    print(np.asarray(clean_arr))
    
    #once you have the data, calculate average travel times

In [155]:
main()

0 at the stop
passed the stop-79.451202,-79.4713359,,-79.463586
passed the stop-79.3398969,-79.358437,,-79.3489
passed the stop-79.3251499,-79.339882,,-79.33896
passed the stop-79.363518,-79.347748,,-79.3489
passed the stop-79.341454,-79.32682,,-79.33896
passed the stop-79.465919,-79.441383,,-79.463586
passed the stop-79.467613,-79.43895,,-79.463586
(63, 6)
[[  1.00400000e+03   4.36411820e+01  -7.94341350e+01   2.52000000e+02
    1.51492442e+09   4.00000000e+00]
 [  1.00400000e+03   4.36388170e+01  -7.94458310e+01   2.42000000e+02
    1.51492474e+09   4.00000000e+00]
 [  1.01700000e+03   4.36030350e+01  -7.94947970e+01   2.41000000e+02
    1.51492474e+09   4.00000000e+00]
 [  1.01900000e+03   4.35926020e+01  -7.95423660e+01   2.51000000e+02
    1.51492474e+09   4.00000000e+00]
 [  1.20000000e+03   4.36342010e+01  -7.94263150e+01   7.40000000e+01
    1.51492474e+09   0.00000000e+00]
 [  1.36100000e+03   4.36538100e+01  -7.94635860e+01   2.73000000e+02
    1.51492461e+09   0.00000000e+00